In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BookRecommendation").getOrCreate()

In [4]:
from pyspark.ml import NearestNeighbors

ImportError: cannot import name 'NearestNeighbors' from 'pyspark.ml' (C:\Users\johnp\.conda\envs\tf-new\lib\site-packages\pyspark\ml\__init__.py)

In [4]:
spark

In [44]:
import pandas as pd

# Load the pickle files
with open('Exports/book_names.pkl', 'rb') as f:
    book_names = pd.read_pickle(f)

with open('Exports/book_pivot.pkl', 'rb') as f:
    book_pivot = pd.read_pickle(f)

with open('Exports/final_rating.pkl', 'rb') as f:
    final_rating = pd.read_pickle(f)

with open('Exports/model.pkl', 'rb') as f:
    model = pd.read_pickle(f)

In [45]:
book_names

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [46]:
type(book_names)

pandas.core.indexes.base.Index

In [49]:
book_names_df = book_names.to_frame(index=False).reset_index(drop=True)
book_names_df.columns = ['title']

In [50]:
type(book_names_df)

pandas.core.frame.DataFrame

In [51]:
book_names_df

,title
0,1984
1,1st to Die: A Novel
2,2nd Chance
3,4 Blondes
4,84 Charing Cross Road
...,...
737,Year of Wonders
738,You Belong To Me
739,Zen and the Art of Motorcycle Maintenance: An ...
740,Zoya


In [47]:
type(final_rating)

pandas.core.frame.DataFrame

In [43]:
final_rating

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
...,...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50


In [52]:
# Merge 'book_names' and 'final_rating' to get book_pivot
book_pivot = pd.merge(book_names_df, final_rating[['title', 'user_id', 'rating']], on='title', how='inner')

In [53]:
book_pivot

,title,user_id,rating
0,1984,254,9
1,1984,11676,0
2,1984,12538,0
3,1984,15408,9
4,1984,16795,8
...,...,...,...
59845,"\O\"" Is for Outlaw""",234359,0
59846,"\O\"" Is for Outlaw""",234623,0
59847,"\O\"" Is for Outlaw""",236283,10
59848,"\O\"" Is for Outlaw""",238120,0


In [54]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

# Assuming book_pivot is a pandas DataFrame
book_pivot_spark = spark.createDataFrame(book_pivot.reset_index())

In [55]:
book_pivot_spark

DataFrame[index: bigint, title: string, user_id: bigint, rating: bigint]

In [56]:
from pyspark import broadcast

In [58]:
# Assuming book_name is a string
broadcast_book_name = broadcast(spark.createDataFrame([(book_names_df,)], ["title"]))

PySparkTypeError: [CANNOT_INFER_TYPE_FOR_FIELD] Unable to infer the type of the field `title`.